# Areal Source models for Nath & Thingbaijam (2012)

Read the source description input files from the online supplementary
material and write them to XML.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys

import numpy as np
import pandas as pd
import geopandas as gpd
from IPython.display import display
from IPython import get_ipython
from time import sleep

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from shapely.geometry import Point, Polygon

from obspy.imaging.beachball import beachball, aux_plane

from openquake.hazardlib import tom, geo
import openquake.hmtk.sources as src
from openquake.hmtk.plotting.mapping import HMTKBaseMap
from openquake.hmtk.parsers.source_model.nrml04_parser import \
    nrmlSourceModelParser

In [3]:
sys.path.append('../utilities')
import source_model_tools as smt  # noqa
from toolbox import PseudoCatalogue, wrap, annotate  # noqa

In [4]:
# define some lists needed at different stages
min_mags = [4.5, 5.5]

layer_ids = [1, 2, 3, 4]
layer_depths_km = [0., 25., 70., 180., 300.]

layers_df = pd.DataFrame(list(zip(layer_ids,
                                  layer_depths_km[:-1],
                                  layer_depths_km[1:])),
                         columns=['id', 'zmin', 'zmax'])
layers_df['id'] = layers_df['id'].astype(int)

layers_df

,id,zmin,zmax
0,1,0.0,25.0
1,2,25.0,70.0
2,3,70.0,180.0
3,4,180.0,300.0


In [5]:
# define the input file names from the original paper
model_path = '../Data/nath2012probabilistic'
polygon_file_template = os.path.join(model_path, 'polygonlay%d.txt')
seismicity_file_template = os.path.join(model_path, 'seismicitylay%d.txt')
polygon_files = [polygon_file_template % i for i in layer_ids]
seismicity_files = [seismicity_file_template % i for i in layer_ids]

# implement some column-name replacements
variable_mapping = {'avalue': 'a', 'bvalue': 'b', 'stdbvalue': 'stdb'}

# an input file supplies some auxiliary data
aux_file = 'auxiliary data.csv'

# define prefixes for the output file names and models
areal_source_model_file = 'areal_source_model'

In [7]:
# read areal polygons and seismicity statistics for each layer
areal_df = pd.DataFrame()
areal_polygons_df = pd.DataFrame()
for i, layer in layers_df.iterrows():

    # read seismicity and polygons
    layer_seis_df = pd.read_csv(seismicity_files[i])
    layer_seis_df.rename(columns=variable_mapping, inplace=True)
    layer_poly_df = read_polygons(polygon_files[i])

    # fill in depths, specify source mechanisms, clean up dip & rake
    n_zones = len(layer_seis_df)
    idx = layer_seis_df.index
    layer_seis_df['zmin'] = pd.Series(np.full(n_zones, layer['zmin']),
                                      index=idx)
    layer_seis_df['zmax'] = pd.Series(np.full(n_zones, layer['zmax']),
                                      index=idx)
    layer_seis_df['layerid'] = pd.Series(np.full(n_zones, layer['id']),
                                         index=idx)

    # put it all together
    layer_source_df = pd.merge(layer_seis_df, layer_poly_df, on='zoneid')
    areal_df = pd.concat([areal_df, layer_source_df], ignore_index=True)
    areal_polygons_df = pd.concat([areal_polygons_df, layer_poly_df],
                                  ignore_index=True)

In [8]:
# merge auxiliary data (crucially, tectonic zones)
aux_df = pd.read_csv(aux_file)
aux_df = aux_df.drop(['zmin', 'zmax', 'dip', 'rake', 'mechanism'], axis=1)
areal_df = pd.merge(areal_df, aux_df, on='zoneid')

# convert polygons coordinates to objects
areal_df['polygon'] = [
    smt.MyPolygon([geo.point.Point(lat, lon)
                   for lat, lon in area_series['polygon coordinates']])
    for _, area_series in areal_df.iterrows()]

# TODO: see if format needed for geopandas can be for openquake objects
areal_df['geometry'] = [Polygon(coordinates) 
                        for coordinates in areal_df['polygon coordinates']]
(areal_df['centroid longitude'], 
 areal_df['centroid latitude'])= zip(*[
     [item[0] for item in geometry.centroid.coords.xy]
     for geometry in areal_df['geometry']])

# convert zoneid to string - sort_and_reindex takes care of "human" sorting
areal_df['zoneid'] = [str(item) for item in areal_df['zoneid']]
areal_df = smt.sort_and_reindex(areal_df)

In [9]:
# add some information to tables
areal_df['rake'] = wrap(areal_df['rake'])
areal_df['mechanism'] = smt.focal_mechanism(areal_df['dip'], areal_df['rake'])
areal_df['new style'] = smt.faulting_style(areal_df['strike'], areal_df['dip'], areal_df['rake'])
areal_df['strike2'],areal_df['dip2'], areal_df['rake2'] = zip(*[
    aux_plane(strike, dip, rake) 
    for strike, dip, rake in zip(
        areal_df['strike'], areal_df['dip'], areal_df['rake'])])          
areal_df['mechanism2'] = smt.focal_mechanism(areal_df['dip2'], areal_df['rake2'])
areal_df.loc[areal_df['dip'] == -1, 'dip'] = np.nan
areal_df.loc[areal_df['mechanism'] == 'undefined', 'rake'] = np.nan
areal_df.loc[areal_df['strike'] == -1, 'strike'] = np.nan

areal_df['mmin'] = min_mags[0]

areal_df['strike2'] = areal_df['strike2'].apply(lambda x: round(x, 1))
areal_df['dip2'] = areal_df['dip2'].apply(lambda x: round(x, 1))
areal_df['rake2'] = areal_df['rake2'].apply(lambda x: round(x, 1))

In [10]:
areal_df[areal_df['zoneid'] == '936'].squeeze()

zoneid                                                               936
layerid                                                                3
tectonic subregion                        subduction intraslab Himalayas
a                                                                   5.25
aspect ratio                                                           2
b                                                                   1.24
centroid latitude                                                29.4592
centroid longitude                                               84.9706
concerns                                                             NaN
dip                                                                   47
geometry               POLYGON ((79.78 32.46, 83.93000000000001 29.84...
mmax                                                                 6.7
msr                                                    StrasserIntraslab
polygon                <source_model_tools.MyPolygo

In [11]:
# read in additional information provided by Kiran Thingbaijam
aux2_df = pd.read_csv('TRT_assignments_KKST.csv', na_values=['nan'], keep_default_na=False, )
aux2_df['zoneid'] = aux2_df['zoneid'].astype(str)

In [12]:
# merge this info with the areal zone table
areal2_df = pd.merge(areal_df, aux2_df, 
                     on=['zoneid', 'layerid'], how='left')
areal2_df.fillna('', inplace=True)
areal2_df.sort_values(['layerid', 'zoneid'], 
                      ascending=[False, True], inplace=True)

In [13]:
duplicated_df = areal2_df[
    areal2_df.duplicated(['strike', 'dip', 'rake'], keep=False) & 
    ~pd.isnull(areal2_df['dip']) &
    (areal2_df['mechanism'] != 'undefined')].copy()
duplicated_df.sort_values('dip', inplace=True)
duplicated_df[
    ['zoneid', 'layerid', 'faulting style', 'new style',
     'strike', 'dip', 'rake', 'mechanism', 
     'strike2', 'dip2', 'rake2', 'mechanism2']
]

,zoneid,layerid,faulting style,new style,strike,dip,rake,mechanism,strike2,dip2,rake2,mechanism2
96,933,2.0,reverse,reverse,283,26,129,reverse,61.0,70.1,72.9,reverse
84,921,1.0,reverse,reverse,283,26,129,reverse,61.0,70.1,72.9,reverse
88,925,2.0,normal,normal,65,37,-26,sinistral,176.3,74.7,-124.1,normal
90,927,2.0,normal,normal,65,37,-26,sinistral,176.3,74.7,-124.1,normal
46,119,2.0,reverse,reverse,112,40,90,reverse,292.0,50.0,90.0,reverse
76,912,1.0,reverse,reverse,112,40,90,reverse,292.0,50.0,90.0,reverse
78,914,1.0,strike-slip,reverse,192,46,124,reverse,327.8,53.4,59.9,reverse
80,916,1.0,reverse,reverse,192,46,124,reverse,327.8,53.4,59.9,reverse
33,93,2.0,strike-slip,strike-slip,196,77,-2,sinistral,286.5,88.1,-167.0,sinistral
36,97,2.0,strike-slip,strike-slip,196,77,-2,sinistral,286.5,88.1,-167.0,sinistral


In [14]:
# bring terminologies in line for comparison
areal2_df['tectonic subregion'] = (
    areal2_df['tectonic subregion']
    .str.lower()
    .str.replace('himalayas', '')
    .str.replace('strike-slip reverse', '')
    .str.replace('normal', '')
    .str.strip()
)                                
areal2_df['tectonic zone'] = (
    areal2_df['tectonic zone']
    .str.lower()
)
areal2_df['mechanism'] = (
    areal2_df['mechanism']
    .replace('dextral', 'strike-slip')
    .replace('sinistral', 'strike-slip')
)
areal2_df['mechanism2'] = (
    areal2_df['mechanism2']
    .replace('dextral', 'strike-slip')
    .replace('sinistral', 'strike-slip')
)

different_mechanism_df = areal2_df[
    (areal2_df['mechanism'] != areal2_df['faulting style']) &
    (areal2_df['tectonic subregion'] != 'no seismicity') &
    (areal2_df['dip'].apply(np.isreal))
]
different_trt_df = areal2_df[
    (areal2_df['tectonic subregion'] != areal2_df['tectonic region type']) &
    (areal2_df['tectonic subregion'] != 'no seismicity')
]

print('%d/%d (%d%%) mechanisms different' % (len(different_mechanism_df), 
                                             len(areal_df), 
                                             100*len(different_mechanism_df)/len(areal_df)))
print('%d/%d (%d%%) TRTs different' % (len(different_trt_df), 
                                  len(areal_df), 
                                  100*len(different_trt_df)/len(areal_df)))


14/104 (13%) mechanisms different
47/104 (45%) TRTs different


In [45]:
%matplotlib inline
subset_df = areal2_df[
    areal2_df['dip'].apply(np.isreal) & 
    (areal2_df['dip'] != -1) &
    (areal2_df['faulting style'] != '')]
wrong_df = different_mechanism_df[
    different_mechanism_df['dip'].apply(np.isreal) & 
    (different_mechanism_df['dip'] != -1) &
    (different_mechanism_df['faulting style'] != '')]
colours = {
    'normal': 'red', 
    'reverse': 'green', 
    'strike-slip': 'blue'
}

fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

for name, group in subset_df.groupby('faulting style'):
    ax.plot(group['rake'], group['dip'], color=colours[name],
            marker='o', markersize=15, linestyle='', linewidth=2, 
            label=name)
    for x, y, zone_id in zip(group['rake'], group['dip'], group['zoneid']):
        ax.annotate(s=zone_id, xy=(x, y), 
                    xytext=(0, 0), textcoords='offset points', 
                    fontsize=6, fontweight='bold', color='white',
                    horizontalalignment='center', 
                    verticalalignment='center')
for _, zone in wrong_df.iterrows():
    ax.plot([zone['rake'], zone['rake2']], [zone['dip'], zone['dip2']], linestyle=':',
            color='black', linewidth=0.5, label=None)   
    ax.plot(zone['rake'], zone['dip'], color=colours[zone['mechanism']],
            marker='o', markersize=10, markeredgecolor='black', 
            linestyle='', linewidth=2,  label=None)
    ax.plot(zone['rake2'], zone['dip2'], color=colours[zone['mechanism2']],
            marker='o', markersize=10, markeredgecolor='black', 
            linestyle='', linewidth=2,  label=None)

ax.legend(loc='lower left')
for threshhold in [-150, -30, 30, 150]:
    ax.axvline(x=threshhold, linestyle='--', color='grey', linewidth=0.5)
ax.set_xlim((-180, 180))
ax.set_xlabel(('Rake [°]'))
ax.set_xticks(range(-180, 181, 45))
ax.set_ylim((0, 90))
ax.set_ylabel(('Dip [°]'))
ax.set_yticks(range(0, 91, 45))

fig.savefig('FocalMechanisms.pdf', transparent=True, 
            bbox_inches='tight', pad_inches=0.1)
plt.close(fig)  # uncomment to view in notebook

In [16]:
wrong_df[
    ['zoneid', 'layerid', 'faulting style', 'new style', 
     'strike', 'dip', 'rake', 'mechanism', 
     'strike2', 'dip2', 'rake2', 'mechanism2']]

,zoneid,layerid,faulting style,new style,strike,dip,rake,mechanism,strike2,dip2,rake2,mechanism2
62,169,4.0,reverse,normal,305,81,-113,normal,194.8,24.6,-22.1,strike-slip
63,170,4.0,normal,reverse,103,67,92,reverse,277.9,23.1,85.3,reverse
57,162,3.0,strike-slip,reverse,176,58,147,reverse,285.0,62.5,36.7,reverse
98,936,3.0,reverse,normal,140,47,-164,strike-slip,38.9,78.4,-44.1,normal
45,118,2.0,strike-slip,normal,301,89,-141,normal,210.2,51.0,-1.3,strike-slip
50,131,2.0,normal,normal,256,43,-12,strike-slip,354.8,81.8,-132.4,normal
31,91,2.0,reverse,reverse,63,33,16,strike-slip,319.5,81.4,122.0,reverse
88,925,2.0,normal,normal,65,37,-26,strike-slip,176.3,74.7,-124.1,normal
90,927,2.0,normal,normal,65,37,-26,strike-slip,176.3,74.7,-124.1,normal
91,928,2.0,strike-slip,normal,301,89,-141,normal,210.2,51.0,-1.3,strike-slip


In [17]:
bb_dir = './beachballs/'

if not os.path.isdir(bb_dir):
    os.mkdir(bb_dir)

for _, event in areal_df.iterrows():
    fig = plt.figure(1)
    focal_mechanism = (event['strike'], event['dip'], event['rake'])
    file_name = os.path.join(bb_dir, zone['zoneid'] + '.svg')
    beachball(focal_mechanism, outfile=file_name, fig=fig, facecolor='black')

In [18]:
df_events = pd.read_csv('../Data/nath2011peak/Table6.csv', parse_dates=['Date'], na_values='-')
df_events.sort_values('H (km)', ascending=True, inplace=True)
df_events[(df_events['H (km)'] >= 180) & (df_events['H (km)'] < 600)]

,Date,M ($M_W$),Lat (°N),Lon (°E),φ,δ,Λ,H (km),FT,RL (km),RW (km),P,N,Ref
28,2005-12-12,6.5,36.45,71.06,279.0,40.0,106.0,210.2,R,20.0,18.0,D,7,7.0
23,2002-03-03,7.3,36.57,70.42,282.0,22.0,85.0,228.5,R,57.0,35.0,D,8,7.0


In [19]:
bb_dir = '../Data/nath2011peak/'

if not os.path.isdir(bb_dir):
    os.mkdir(bb_dir)

for _, event in df_events.iterrows():
    fig = plt.figure(1)
    focal_mechanism = (event['φ'], event['δ'], event['Λ'])
    file_name = os.path.join(bb_dir, str(event['Date'].date()).replace('-', '') + '.svg')
    beachball(focal_mechanism, outfile=file_name, fig=fig, facecolor='blue')

In [20]:
different_trt_df[[
    'zoneid', 'layerid', 'zmin', 'zmax', 'strike', 'dip', 'rake', 'a',
    'mechanism', 'faulting style', 
    'tectonic zone', 'tectonic subregion', 'tectonic region type'
]]

,zoneid,layerid,zmin,zmax,strike,dip,rake,a,mechanism,faulting style,tectonic zone,tectonic subregion,tectonic region type
62,169,4.0,180.0,300.0,305,81,-113,4.78,normal,reverse,subduction intraslab,subduction intraslab,subduction interface
55,153,3.0,70.0,180.0,247,44,94,6.13,reverse,reverse,subduction intraslab,subduction intraslab,subduction interface
56,154,3.0,70.0,180.0,100,80,-169,6.37,strike-slip,strike-slip,subduction intraslab,subduction intraslab,
58,163,3.0,70.0,180.0,283,46,55,5.60,reverse,reverse,subduction intraslab,subduction intraslab,subduction interface
60,166,3.0,70.0,180.0,270,61,46,5.02,reverse,reverse,subduction intraslab,subduction intraslab,subduction interface
61,167,3.0,70.0,180.0,206,49,46,5.41,reverse,reverse,subduction intraslab,subduction intraslab,subduction interface
97,935,3.0,70.0,180.0,317,57,111,4.77,reverse,reverse,subduction intraslab,subduction intraslab,subduction interface
98,936,3.0,70.0,180.0,140,47,-164,5.25,strike-slip,reverse,subduction intraslab,subduction intraslab,subduction intraslab
39,100,2.0,25.0,70.0,89,89,-13,4.76,strike-slip,strike-slip,subduction interface,subduction interface,
40,101,2.0,25.0,70.0,349,40,-117,5.28,normal,normal,subduction interface,subduction interface,


In [21]:
different_mechanism_df[[
    'zoneid', 'layerid', 'zmin', 'zmax', 'strike', 'dip', 'rake', 
    'mechanism', 'faulting style', 
]]

,zoneid,layerid,zmin,zmax,strike,dip,rake,mechanism,faulting style
62,169,4.0,180.0,300.0,305,81,-113,normal,reverse
63,170,4.0,180.0,300.0,103,67,92,reverse,normal
57,162,3.0,70.0,180.0,176,58,147,reverse,strike-slip
98,936,3.0,70.0,180.0,140,47,-164,strike-slip,reverse
45,118,2.0,25.0,70.0,301,89,-141,normal,strike-slip
50,131,2.0,25.0,70.0,256,43,-12,strike-slip,normal
31,91,2.0,25.0,70.0,63,33,16,strike-slip,reverse
88,925,2.0,25.0,70.0,65,37,-26,strike-slip,normal
90,927,2.0,25.0,70.0,65,37,-26,strike-slip,normal
91,928,2.0,25.0,70.0,301,89,-141,normal,strike-slip


In [22]:
different_mechanism_df[different_mechanism_df['faulting style'] != different_mechanism_df['new style']][[
    'zoneid', 'layerid', 'zmin', 'zmax', 'strike', 'dip', 'rake', 
    'mechanism', 'mechanism2', 'faulting style', 'new style', 
]]

,zoneid,layerid,zmin,zmax,strike,dip,rake,mechanism,mechanism2,faulting style,new style
62,169,4.0,180.0,300.0,305,81,-113,normal,strike-slip,reverse,normal
63,170,4.0,180.0,300.0,103,67,92,reverse,reverse,normal,reverse
57,162,3.0,70.0,180.0,176,58,147,reverse,reverse,strike-slip,reverse
98,936,3.0,70.0,180.0,140,47,-164,strike-slip,normal,reverse,normal
45,118,2.0,25.0,70.0,301,89,-141,normal,strike-slip,strike-slip,normal
91,928,2.0,25.0,70.0,301,89,-141,normal,strike-slip,strike-slip,normal
11,14,1.0,0.0,25.0,228,69,-30,strike-slip,strike-slip,reverse,strike-slip
20,66,1.0,0.0,25.0,58,83,25,strike-slip,strike-slip,reverse,strike-slip
78,914,1.0,0.0,25.0,192,46,124,reverse,reverse,strike-slip,reverse


In [23]:
aux_df[aux_df['tectonic subregion'] == 'no seismicity']

,zoneid,aspect ratio,msr,tectonic zone,region,tectonic subregion,concerns
22,71,2,WC1994,Oceanic crust,Bay of Bengal & Arabian Sea,no seismicity,NaN
23,86,2,WC1994,Stable continental,"Sri Lanka, Laccadive Sea",no seismicity,NaN
24,9031,2,WC1994,Oceanic crust,Murray Ridge,no seismicity,NaN
25,9081,2,WC1994,Subduction interface,Himalayas,no seismicity,NaN
26,9131,2,WC1994,Subduction interface,Indo-Myanmar,no seismicity,NaN
27,9151,2,WC1994,Oceanic crust,Bay of Bengal,no seismicity,NaN
28,9171,2,WC1994,Oceanic crust,Bay of Bengal,no seismicity,NaN


In [24]:
areal_df[areal_df['stdb'] != 0]['stdb'].describe()

count    99.000000
mean      0.095354
std       0.024839
min       0.030000
25%       0.080000
50%       0.100000
75%       0.115000
max       0.160000
Name: stdb, dtype: float64

In [25]:
# show a summary including megathrust zones and bin statistics
drop_columns = ['tectonic zone', 'region', 'concerns', 'zmax', 'zmin',
                'polygon coordinates', 'polygon', 'geometry', 
                'aspect ratio', 'dip', 'rake', 'strike']
display(pd.concat([areal_df.drop(drop_columns, axis=1).head(),
                   areal_df.drop(drop_columns, axis=1).tail()]))

,zoneid,layerid,tectonic subregion,a,b,centroid latitude,centroid longitude,mmax,msr,stdb,stdmmax,mechanism,new style,strike2,dip2,rake2,mechanism2,mmin
0,1,1.0,subduction interface,5.20,1.05,38.794988,70.440638,8.0,StrasserInterface,0.07,0.4,dextral,strike-slip,102.4,61.4,160.6,dextral,4.5
1,2,1.0,subduction interface,4.64,0.93,39.234824,74.792116,8.0,StrasserInterface,0.09,0.4,reverse,reverse,69.8,71.5,64.7,reverse,4.5
2,3,1.0,active shallow crust strike-slip reverse,3.75,0.77,39.433139,77.662187,8.0,WC1994,0.05,0.4,reverse,reverse,328.4,57.0,178.8,dextral,4.5
3,4,1.0,subduction interface,4.34,0.98,37.458307,80.409876,7.4,StrasserInterface,0.08,0.4,dextral,reverse,40.7,87.5,34.0,reverse,4.5
4,5,1.0,subduction interface,4.29,0.91,36.838544,68.780604,7.8,StrasserInterface,0.08,0.4,reverse,reverse,23.7,64.3,39.5,reverse,4.5
99,9031,2.0,no seismicity,0.00,0.00,22.759109,63.890879,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
100,9081,2.0,no seismicity,0.00,0.00,29.074076,95.649902,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
101,9131,2.0,no seismicity,0.00,0.00,22.041780,91.829942,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
102,9151,2.0,no seismicity,0.00,0.00,16.456705,92.439141,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
103,9171,2.0,no seismicity,0.00,0.00,9.733105,88.684630,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5


In [26]:
areal_df[areal_df['tectonic subregion'] == 'no seismicity'].drop(drop_columns,
                                                                 axis=1)

,zoneid,layerid,tectonic subregion,a,b,centroid latitude,centroid longitude,mmax,msr,stdb,stdmmax,mechanism,new style,strike2,dip2,rake2,mechanism2,mmin
21,71,1.0,no seismicity,0.0,0.81,9.880466,77.972208,5.8,WC1994,0.05,0.3,undefined,undefined,270.0,90.0,89.0,reverse,4.5
26,86,1.0,no seismicity,0.0,0.81,6.785345,78.740244,6.3,WC1994,0.05,0.2,undefined,undefined,270.0,90.0,89.0,reverse,4.5
99,9031,2.0,no seismicity,0.0,0.00,22.759109,63.890879,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
100,9081,2.0,no seismicity,0.0,0.00,29.074076,95.649902,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
101,9131,2.0,no seismicity,0.0,0.00,22.041780,91.829942,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
102,9151,2.0,no seismicity,0.0,0.00,16.456705,92.439141,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
103,9171,2.0,no seismicity,0.0,0.00,9.733105,88.684630,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5


In [44]:
props = ['a', 'b', 'mmax']
ranges = [np.arange(-0.5, 8, 1),
          np.arange(-0.1, 1.7, 0.2),
          np.arange(-0.5, 10, 1)]
groups = areal_df.groupby('layerid')
fig, axes = plt.subplots(nrows=len(props), ncols=1, figsize=(6, 3*len(props)))
for prop, ax, bins in zip(props, axes, ranges):
    data = [group[prop] for _, group in groups]
    labels = ['layer %d' % layer_id for layer_id, _ in groups]
    ax.hist(data, label=labels, stacked=True, bins=bins)
    ax.set_ylabel(prop)
axes[0].legend(loc='upper left')
fig.savefig("ArealModelFmds.pdf", dpi=300,
            transparent=True, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)  # uncomment to view in notebook

In [28]:
print(set(areal_df['tectonic subregion']))
areal_df[areal_df['tectonic subregion'] == 'no seismicity'].drop(drop_columns,
                                                                 axis=1)

{'stable shallow crust', 'subduction intraslab', 'subduction intraslab Himalayas', 'active shallow crust normal', 'active shallow crust strike-slip reverse', 'intraplate margin upper', 'intraplate margin lower', 'no seismicity', 'subduction interface'}


,zoneid,layerid,tectonic subregion,a,b,centroid latitude,centroid longitude,mmax,msr,stdb,stdmmax,mechanism,new style,strike2,dip2,rake2,mechanism2,mmin
21,71,1.0,no seismicity,0.0,0.81,9.880466,77.972208,5.8,WC1994,0.05,0.3,undefined,undefined,270.0,90.0,89.0,reverse,4.5
26,86,1.0,no seismicity,0.0,0.81,6.785345,78.740244,6.3,WC1994,0.05,0.2,undefined,undefined,270.0,90.0,89.0,reverse,4.5
99,9031,2.0,no seismicity,0.0,0.00,22.759109,63.890879,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
100,9081,2.0,no seismicity,0.0,0.00,29.074076,95.649902,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
101,9131,2.0,no seismicity,0.0,0.00,22.041780,91.829942,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
102,9151,2.0,no seismicity,0.0,0.00,16.456705,92.439141,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
103,9171,2.0,no seismicity,0.0,0.00,9.733105,88.684630,0.0,WC1994,0.00,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5


In [43]:
catalogue_df = pd.read_csv('../Catalogue/SACAT1900_2008v2.txt', sep='\t')

fig, ax = plt.subplots(figsize=(6, 6))
catalogue_df['SHOCK_TYPE'].value_counts().plot(kind='pie', ax=ax)
fig.savefig('ShockTypes.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)  # uncomment to view in notebook

In [30]:
# augment catalogue with zone and layer info
catalogue_df['geometry'] = [Point(lon, lat) 
                            for lon, lat in zip(catalogue_df['LON'], 
                                                catalogue_df['LAT'])]
layer_catalogue_gdfs = []
for _, layer in layers_df.iterrows():
    layer_catalogue_gdf = gpd.GeoDataFrame(catalogue_df[
        (catalogue_df['DEPTH'] >= layer['zmin']) &
        (catalogue_df['DEPTH'] < layer['zmax'])], crs='WGS84')
    layer_areal_gdf = gpd.GeoDataFrame(areal_df[
        areal_df['layerid'] == layer['id']][['geometry', 'zoneid', 'layerid']], crs='WGS84')
    layer_catalogue_gdfs.append(gpd.sjoin(layer_catalogue_gdf, layer_areal_gdf, 
                                          how='left', op='intersects'))
catalogue_df = pd.concat(layer_catalogue_gdfs).drop('geometry')

In [31]:
pd.concat((catalogue_df.head(), catalogue_df.tail()))

,YEAR,MONTH,DAY,HH,MM,SS,LAT,LON,DEPTH,DEPTH_ERR,...,MAG,MAG_ERR,MAG_MW,MAG_MW_ERR,SOURCE,SHOCK_TYPE,geometry,index_right,zoneid,layerid
39,1906,10,4,6,52,-1.00,37.2000,67.3000,20.0,-1.0,...,5.70,-1.0,5.8,-1.00,AB2003a,Mainshock,POINT (67.3 37.2),NaN,NaN,NaN
46,1907,10,21,4,23,36.00,38.7000,68.1000,24.0,-1.0,...,7.30,-1.0,7.2,-1.00,UL2006a,Mainshock,POINT (68.09999999999999 38.7),NaN,NaN,NaN
48,1907,10,24,5,10,-1.00,38.0000,68.8000,18.0,-1.0,...,5.20,-1.0,5.5,-1.00,AB2003a,Mainshock,POINT (68.8 38),0.0,1,1.0
74,1911,1,1,14,59,0.00,36.5000,66.0000,20.0,-1.0,...,6.47,-1.0,6.4,-1.00,AB2003a,Mainshock,POINT (66 36.5),NaN,NaN,NaN
96,1913,9,9,18,43,-1.00,37.1000,68.1000,18.0,-1.0,...,5.00,-1.0,5.4,-1.00,AB2003a,Mainshock,POINT (68.09999999999999 37.1),4.0,5,1.0
50295,2008,12,26,15,56,29.97,37.1880,72.0040,216.8,41.0,...,3.20,0.1,3.4,0.28,ISC,Mainshock,POINT (72.00399999999999 37.188),63.0,170,4.0
50297,2008,12,26,22,12,56.66,36.2475,70.3301,189.1,66.0,...,3.20,0.2,3.4,0.33,ISC,Mainshock,POINT (70.3301 36.2475),62.0,169,4.0
50317,2008,12,29,18,24,32.58,36.4481,71.0448,200.0,46.0,...,3.30,0.1,3.5,0.28,ISC,Aftershock,POINT (71.0448 36.4481),62.0,169,4.0
50319,2008,12,30,3,9,28.10,36.4556,70.9448,198.7,29.0,...,3.90,-1.0,4.6,0.19,ISC,Aftershock,POINT (70.9448 36.4556),62.0,169,4.0
50331,2008,12,31,20,48,35.71,36.2467,70.8264,197.0,44.0,...,3.30,0.1,3.5,0.28,ISC,Mainshock,POINT (70.82640000000001 36.2467),62.0,169,4.0


In [32]:
# read completeness table in anticipation of computing activity rates from catalogues
completeness_df = pd.read_csv('../Data/thingbaijam2011seismogenic/Table1.csv',
                              header=[0, 1], index_col=[0, 1])
# completeness_df.reset_index(inplace=True)
completeness_df.columns = [' '.join(col).strip()
                           for col in completeness_df.columns.values]
completeness_df.reset_index(inplace=True)
display(completeness_df)

,zmin,zmax,4 start,4 end,4.5 start,4.5 end,5.5 start,5.5 end
0,0,25,1994,2008,1964,2008,1903,2008
1,25,70,1990,2008,1964,2008,1902,2008
2,70,180,1996,2008,1964,2008,1914,2008
3,180,300,1970,2008,1984,2008,1912,2008


In [33]:
# for each minimum magnitude and layer work out the activity rates
catalogue_activity_df = pd.DataFrame()
for _, layer in pd.merge(completeness_df, layers_df).iterrows():
    layer_results = pd.Series()
    for mag in reversed(min_mags):
        above_thresh = catalogue_df['MAG_MW'] >= mag
        start = layer[str(mag) + ' start']
        end = layer[str(mag) + ' end']
        at_depth = ((catalogue_df['DEPTH'] >= layer['zmin']) &
                    (catalogue_df['DEPTH'] < layer['zmax']))
        in_years = ((catalogue_df['YEAR'] >= start) &
                    (catalogue_df['YEAR'] <= end))
        in_a_zone = catalogue_df['zoneid'] != -1
        is_mainshock = catalogue_df['SHOCK_TYPE'] == 'Mainshock'
        subcat_df = catalogue_df[
            above_thresh & at_depth & in_years & in_a_zone & is_mainshock]
        layer_results = layer_results.append(pd.Series({
            'catalogue ' + str(mag):
                round(float(len(subcat_df))/(end - start + 1), 1),
            }, name=layer['id']))
    catalogue_activity_df = catalogue_activity_df.append(layer_results, ignore_index=True)
catalogue_activity_df = catalogue_activity_df.append(pd.Series(
    catalogue_activity_df.sum(axis=0), name='Total'))

In [34]:
areal2_df.to_csv(areal_source_model_file + '.csv', index=False)

In [35]:
# write areal model data to TSV file ==> 
# when this completes, logic trees and smoothed seismicity models can be generated
areal_output_df = smt.sort_and_reindex(smt.add_name_id(
    smt.twin_source_by_magnitude(areal_df)).drop(['polygon', 'polygon coordinates'], axis=1))
areal_output_df.to_csv(areal_source_model_file + '.tsv', sep='\t', index=False, float_format='%.12g')
areal_output_df = smt.sort_and_reindex(smt.add_name_id(
    areal_df).drop(['polygon', 'geometry'], axis=1))
areal_output_df.to_csv(areal_source_model_file + '_no_twin.tsv',
                       sep='\t', index=False, float_format='%.12g')

In [36]:
# write each layer of areal source model to KML with added binwise rates
areal_kml_df = smt.add_name_id(
    smt.add_binwise_rates(areal_df).drop(['polygon', 'geometry'], axis=1))
for layer_id in layer_ids:
    this_layer = areal_kml_df['layerid'] == layer_id
    temp_df = areal_kml_df.drop(['layerid'], axis=1)
    smt.source_df_to_kml(temp_df.loc[this_layer, :],
                         '%s layer %d' % (areal_source_model_file, layer_id))

../utilities/source_model_tools.py:536: RuntimeWarning: divide by zero encountered in log10
  log_n_bin = np.log10(10**log_n_m_lo - 10**log_n_m_hi).round(2)
../utilities/source_model_tools.py:536: RuntimeWarning: invalid value encountered in log10
  log_n_bin = np.log10(10**log_n_m_lo - 10**log_n_m_hi).round(2)


In [37]:
areal_df[pd.isnull(areal_df['dip'])]

,zoneid,layerid,tectonic subregion,a,aspect ratio,b,centroid latitude,centroid longitude,concerns,dip,...,tectonic zone,zmax,zmin,mechanism,new style,strike2,dip2,rake2,mechanism2,mmin
21,71,1.0,no seismicity,0.00,2,0.81,9.880466,77.972208,NaN,NaN,...,Oceanic crust,25.0,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
23,75,1.0,stable shallow crust,3.91,2,1.19,19.216992,79.463187,NaN,NaN,...,Stable continental,25.0,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
24,81,1.0,stable shallow crust,2.65,2,0.86,15.482657,80.465740,NaN,NaN,...,Stable continental,25.0,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
26,86,1.0,no seismicity,0.00,2,0.81,6.785345,78.740244,NaN,NaN,...,Stable continental,25.0,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
86,923,1.0,stable shallow crust,2.51,2,0.73,12.772866,76.672790,NaN,NaN,...,Stable continental,25.0,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
87,924,1.0,stable shallow crust,1.58,2,0.63,19.633479,83.904819,NaN,NaN,...,Stable continental,25.0,0.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
99,9031,2.0,no seismicity,0.00,2,0.00,22.759109,63.890879,NaN,NaN,...,Oceanic crust,70.0,25.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
100,9081,2.0,no seismicity,0.00,2,0.00,29.074076,95.649902,NaN,NaN,...,Subduction interface,70.0,25.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
101,9131,2.0,no seismicity,0.00,2,0.00,22.041780,91.829942,NaN,NaN,...,Subduction interface,70.0,25.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5
102,9151,2.0,no seismicity,0.00,2,0.00,16.456705,92.439141,NaN,NaN,...,Oceanic crust,70.0,25.0,undefined,undefined,270.0,90.0,89.0,reverse,4.5


In [38]:
# write areal source model with megathrust sources twinned to NRML
areal_source_list = smt.source_df_to_list(
    smt.add_name_id(smt.twin_source_by_magnitude(areal_df)))
areal_source_model = src.source_model.mtkSourceModel(
    identifier='1',
    name=os.path.split(model_path)[1] + ' areal',
    sources=areal_source_list)
areal_source_model.serialise_to_nrml(areal_source_model_file + '.xml')

In [39]:
catalogue = PseudoCatalogue(areal_source_model)

In [42]:
map_config = {"min_lon": 60, "max_lon": 105,
              "min_lat": 0,  "max_lat": 40, "resolution": "l"}
parser = nrmlSourceModelParser(areal_source_model_file + '.xml')

for depth in sorted(list(set(catalogue.data['depth']))):
    basemap = HMTKBaseMap(map_config, '', lat_lon_spacing=5)

    source_model_read = parser.read_file('Areal Source Model')
    selected_sources = [source for source in source_model_read.sources
                        if source.hypo_depth_dist.data[0][1] == depth]
    source_model_read.sources = selected_sources
    selected_catalogue = PseudoCatalogue(source_model_read)

    basemap.add_source_model(source_model_read, overlay=True, area_border='0.5')
    basemap.add_focal_mechanism(selected_catalogue, magnitude=False)
    
    ax = basemap.fig.gca()
    annotate('%g km' % depth, loc='lower left', ax=ax)
    for _, zone in selected_catalogue.data.iterrows():
        x, y = basemap.m(zone.longitude, zone.latitude)
        ax.annotate(s=zone.id.replace('z', ''), xy=(x, y), color='green', 
                    xytext=(0, 10), textcoords='offset points', zorder=100,
                    fontsize=8, fontweight='bold', horizontalalignment='center')
    basemap.fig.savefig('ArealModelFocalMechanisms%gkm.pdf' % depth, transparent=False,
            dpi=300, bbox_inches='tight', pad_inches=0.1)

plt.close('all')  # uncomment to view in notebook